In [16]:
import sys
import os
import time
sys.path.append(".")
sys.path.append("..")

from algorithms.old_path_planning import OldPathPlanner
from algorithms.path_planning import PathPlanner, stanley_smooth_path
from algorithms.optimized_path_planning import stanley_smooth_path as optimized_smooth_path
from algorithms.david_planning import PathPlanner as DavidPathPlanning, stanley_smooth_path as david_smooth

from tvojemama.logger import LogReader

import torch

## Gather data from log

In [2]:
log_path = "../data/bros_logs/04_05_2023-18_29_34/AS/mission_04_05_2023-18_29_34.pkl"
log_reader = LogReader(log_path)

In [3]:
period = 2. # time between percep_data samples in seconds

percep_datas = []

counter = 0
start_time = None
for msg_t, (msg_name, data) in log_reader:
    
    mission_kwargs = data['mission_kwargs']
    percep_data = mission_kwargs['percep_data']
    
    if start_time is None:
        start_time = msg_t
        
    if msg_t / period >= counter:
        counter += 1
        
        if percep_data.shape[0] != 0:
            percep_datas.append(percep_data)
    
print(f"got {len(percep_datas)} samples")

got 3045 samples


## Run tests

In [4]:
def benchmark_path_planning(find_path_fn, smooth_fn=None):
    indiv_times = []
    start_time = time.time()
    for percep_data in percep_datas:
        curr_start_time = time.time()
        path = find_path_fn(percep_data)
        
        if smooth_fn is not None:
            path = smooth_fn(path)
            
        curr_end_time = time.time()
        indiv_times.append(curr_end_time - curr_start_time)
    end_time = time.time()
    
    print(f"took {end_time - start_time:.2f} seconds")
    print(f"\tMean time: {((end_time - start_time) / len(percep_datas) * 1000):.3f} ms")
    print(f"\tMax time: {max(indiv_times) * 1000:.3f} ms")
    print(f"\tMin time: {min(indiv_times) * 1000:.3f} ms")

In [12]:
# initializing
path_planner = PathPlanner()
old_path_planner = OldPathPlanner({"n_steps": 20})
david_planner = DavidPathPlanning()

In [14]:
benchmark_path_planning(find_path_fn=path_planner.find_path, smooth_fn=stanley_smooth_path)

took 34.89 seconds
	Mean time: 11.459 ms
	Max time: 71.688 ms
	Min time: 4.134 ms


In [7]:
benchmark_path_planning(find_path_fn=path_planner.find_path, smooth_fn=None)

took 14.01 seconds
	Mean time: 4.601 ms
	Max time: 17.096 ms
	Min time: 0.785 ms


In [8]:
benchmark_path_planning(find_path_fn=old_path_planner.find_path, smooth_fn=None)

took 2.56 seconds
	Mean time: 0.840 ms
	Max time: 4.201 ms
	Min time: 0.172 ms


In [9]:
benchmark_path_planning(find_path_fn=old_path_planner.find_path, smooth_fn=optimized_smooth_path)

took 20.53 seconds
	Mean time: 6.743 ms
	Max time: 63.550 ms
	Min time: 2.553 ms


In [13]:
benchmark_path_planning(find_path_fn=david_planner.find_path, smooth_fn=david_smooth)

took 19.28 seconds
	Mean time: 6.330 ms
	Max time: 58.989 ms
	Min time: 1.442 ms


In [15]:
benchmark_path_planning(find_path_fn=david_planner.find_path, smooth_fn=None)

took 2.28 seconds
	Mean time: 0.750 ms
	Max time: 13.193 ms
	Min time: 0.317 ms


## Custom solution

In [10]:
def torch_smooth_fn(path):
    pass

In [ ]:
def curvature(waypoints):
    '''
    Curvature as  the sum of the normalized dot product between the way elements
    Implement second term of the smoothing objective.

    args:
        waypoints [2, num_waypoints] !!!!!
    '''
    shift_left = np.roll(waypoints, shift=-1, axis=1)
    shift_right = np.roll(waypoints, shift=1, axis=1)
    left_half = normalize(shift_left - waypoints)
    right_half = normalize(waypoints - shift_right)

    mul = left_half * right_half
    segment = np.sum(mul, axis=0)

    return np.sum(segment[1:-1])

In [ ]:
iters = 1
inp = percep_datas[0]
for it in range(iters):
    # loss fn
    out = objective_fn(inp)
    
    # ground truth -> stara cesta.copy()
    # params & input -> stara cesta.copy()